#Finances 1

In [ ]:
WITH final AS (
SELECT orders.orderNumber, orderDate, status, country, SUM(priceEach*quantityOrdered) AS totalPrice
FROM orders
INNER JOIN customers
ON orders.customerNumber=customers.customerNumber
INNER JOIN orderdetails
ON orderdetails.orderNumber=orders.orderNumber
WHERE 1=1
	AND status != 'cancelled'
	AND 12*year(orderDate) + month(orderDate) = (12*year(current_timestamp) + month(current_timestamp) - 1)
    OR 12*year(orderDate) + month(orderDate) = (12*year(current_timestamp) + month(current_timestamp) - 2)
GROUP BY orderNumber)
SELECT final.country, month(final.orderDate), SUM(final.totalPrice) as turnover
FROM final
GROUP BY month(final.orderDate), final.country
ORDER BY month(final.orderDate), final.country

# Finances 2

In [ ]:
WITH remains AS (
WITH test AS (
SELECT orders.orderNumber, SUM(quantityOrdered*priceEach) as PriceTotal, status, orders.customerNumber
FROM orders
INNER JOIN orderdetails
ON orders.orderNumber=orderdetails.orderNumber
WHERE status !='cancelled'
GROUP BY customerNumber)
SELECT payments.customerNumber, customers.customerName, SUM(amount) as totalPaid, customers.country, customers.salesRepEmployeeNumber,
CONCAT(employees.LastName,' ', employees.firstName) as SalesRepName, employees.reportsTo, CONCAT(e.LastName,' ',e.firstName) as ManagerName, 
test.PriceTotal, test.status
FROM customers
INNER JOIN payments
ON customers.customerNumber=payments.customerNumber
INNER JOIN employees
ON customers.salesRepEmployeeNumber=employees.employeeNumber
INNER JOIN employees e
ON employees.reportsTo=e.employeeNumber
INNER JOIN test
ON test.customerNumber=customers.customerNumber
GROUP BY customers.customerNumber)
SELECT remains.customerNumber, remains.customerName, round(PriceTotal - totalPaid) as totalRemain, remains.country, 
remains.salesRepEmployeeNumber, remains.SalesRepName, remains.reportsTo, remains.ManagerName
FROM remains
WHERE (PriceTotal-totalPaid)>0
ORDER BY customerNumber

# Sales (même code utilisé pour les feuilles F.sales et F.sales2)

In [ ]:
WITH test1 AS (
SELECT productline,
           YEAR(orderDate) order_year2021,
           MONTH(orderDate) order_month2021,
		   SUM(quantityOrdered) order_value2021
    FROM orders
    INNER JOIN orderdetails USING (orderNumber)
    INNER JOIN products USING (productCode)
    WHERE year(orderDate)=YEAR(current_timestamp) and status!='cancelled'
    GROUP BY productline, order_year2021, order_month2021
    ORDER BY order_year2021),
test2 AS (
SELECT productline,
           YEAR(orderDate) order_year2020,
           MONTH(orderDate) order_month2020,
		   SUM(quantityOrdered) order_value2020
    FROM orders
    INNER JOIN orderdetails USING (orderNumber)
    INNER JOIN products USING (productCode)
    WHERE year(orderDate)=YEAR(current_timestamp)-1 and status!='cancelled'
    GROUP BY productline, order_year2020, order_month2020
    ORDER BY order_year2020)

SELECT test1.productline, test1.order_year2021, test2.order_year2020, test1.order_month2021, test2.order_month2020, test1.order_value2021, test2.order_value2020,
ROUND((test1.order_value2021 - test2.order_value2020 )/test2.order_value2020*100, 2) AS ROC
FROM test1
LEFT JOIN test2
ON (test1.productline,test1.order_year2021,test1.order_month2021)=(test2.productline,(test2.order_year2020+1),test2.order_month2020)

#Logistics

In [ ]:
SELECT products.productName, products.productCode, SUM(orderdetails.quantityOrdered), products.quantityInStock FROM products
INNER JOIN orderdetails ON products.productCode = orderdetails.productCode
INNER JOIN orders ON orderdetails.orderNumber = orders.orderNumber
WHERE orders.orderdate BETWEEN '2021-01-01' AND '2021-12-31' 
AND status != 'cancelled'
GROUP BY products.productCode
ORDER BY SUM(orderdetails.quantityOrdered) DESC LIMIT 5

# HR

In [ ]:
SELECT MONTHNAME(orders.orderDate), CONCAT(firstname, ' ', lastname) AS nom_complet, SUM(priceEach*quantityOrdered) AS totalPrice FROM employees

INNER JOIN customers ON customers.salesRepEmployeeNumber = employees.employeeNumber
INNER JOIN orders ON orders.customerNumber = customers.customerNumber
INNER JOIN orderdetails ON orderdetails.orderNumber = orders.orderNumber

WHERE 1=1
AND 12 * YEAR(orderDate) + MONTH(orderDate) = 12 * YEAR(current_timestamp) + MONTH(current_timestamp) - 1
AND orders.status != 'cancelled'
GROUP BY MONTHNAME(orders.orderDate), nom_complet
ORDER BY totalPrice DESC LIMIT 2

# HR Bonus

In [ ]:
WITH test AS (SELECT MONTHNAME(orders.orderDate) as `month`, CONCAT(firstname, ' ', lastname) AS Name, SUM(priceEach*quantityOrdered) AS Turnover FROM employees

INNER JOIN customers ON customers.salesRepEmployeeNumber = employees.employeeNumber  # Jointure employees -> customers
INNER JOIN orders ON orders.customerNumber = customers.customerNumber  # Jointure customers -> orders
INNER JOIN orderdetails ON orderdetails.orderNumber = orders.orderNumber  # Jointure orders -> orderdetails 
WHERE 1=1
AND 12 * YEAR(orderDate) + MONTH(orderDate) = 12 * YEAR(current_timestamp) + MONTH(current_timestamp) 
OR 12 * YEAR(orderDate) + MONTH(orderDate) = 12 * YEAR(current_timestamp) + MONTH(current_timestamp) - 1
OR 12 * YEAR(orderDate) + MONTH(orderDate) = 12 * YEAR(current_timestamp) + MONTH(current_timestamp) - 2 
OR 12 * YEAR(orderDate) + MONTH(orderDate) = 12 * YEAR(current_timestamp) + MONTH(current_timestamp) - 3 
OR 12 * YEAR(orderDate) + MONTH(orderDate) = 12 * YEAR(current_timestamp) + MONTH(current_timestamp) - 4 
OR 12 * YEAR(orderDate) + MONTH(orderDate) = 12 * YEAR(current_timestamp) + MONTH(current_timestamp) - 5 
OR 12 * YEAR(orderDate) + MONTH(orderDate) = 12 * YEAR(current_timestamp) + MONTH(current_timestamp) - 6 
OR 12 * YEAR(orderDate) + MONTH(orderDate) = 12 * YEAR(current_timestamp) + MONTH(current_timestamp) - 7 
OR 12 * YEAR(orderDate) + MONTH(orderDate) = 12 * YEAR(current_timestamp) + MONTH(current_timestamp) - 8 
OR 12 * YEAR(orderDate) + MONTH(orderDate) = 12 * YEAR(current_timestamp) + MONTH(current_timestamp) - 9 
OR 12 * YEAR(orderDate) + MONTH(orderDate) = 12 * YEAR(current_timestamp) + MONTH(current_timestamp) - 10
OR 12 * YEAR(orderDate) + MONTH(orderDate) = 12 * YEAR(current_timestamp) + MONTH(current_timestamp) - 11
AND orders.status != 'cancelled'
AND customers.salesRepEmployeeNumber IS NOT NULL
GROUP BY MONTH(orders.orderDate), Name
ORDER BY MONTH(orders.orderDate), Turnover DESC),

testok AS (
(select * from test where `month` = 'January' order by Turnover DESC limit 2)
union all
(select * from test where `month` = 'February' order by Turnover DESC limit 2)
union all
(select * from test where `month` = 'March' order by Turnover DESC limit 2)
union all
(select * from test where `month` = 'April' order by Turnover DESC limit 2)
union all
(select * from test where `month` = 'May' order by Turnover DESC limit 2)
union all
(select * from test where `month` = 'June' order by Turnover DESC limit 2)
union all
(select * from test where `month` = 'July' order by Turnover DESC limit 2)
union all
(select * from test where `month` = 'August' order by Turnover DESC limit 2)
union all
(select * from test where `month` = 'September' order by Turnover DESC limit 2)
union all
(select * from test where `month` = 'October' order by Turnover DESC limit 2)
union all
(select * from test where `month` = 'November' order by Turnover DESC limit 2)
union all
(select * from test where `month` = 'December' order by Turnover DESC limit 2))

SELECT * FROM testok